In [1]:
__author__ = 'Alice Jacques <alice.jacques@noirlab.edu>, Steve Pothier <steve.pothier@noirlab.edu>, SPARCL team <datalab-spectro@noirlab.edu>'
__version__ = '20240530' # yyyymmdd; 
__datasets__ = ['sdss_dr16', 'boss_dr16', 'desi_edr', 'sdss_dr17_test']

# How to use SPARCL's authentication+authorization feature
Authors: Alice Jacques, Steve Pothier, and the SPARCL team

### Table of contents
* [(1) Login, logout, and check authorized datasets](#loginout)
    - [(1a) A user authorized to access a private dataset](#1a)
    - [(1b) A user unauthorized to access a private dataset](#1b)
    - [(1c) A user that has an SSO account, but is not registered in our Admin](#1c)
    - [(1d) An anonymous user](#1d)
<html><p></html>
* [(2) Auth with the <tt>client.find()</tt> method](#find)
    - [(2a) A user authorized to access a private dataset](#2a)
    - [(2b) A user unauthorized to access a private dataset](#2b)
    - [(2c) A user that has an SSO account, but is not registered in our Admin](#2c)
    - [(2d) An anonymous user](#2d)
<html><p></html>
* [(3) Auth with the <tt>client.retrieve()</tt> method](#retrieve)
    - [(3a) A user authorized to access a private dataset](#3a)
    - [(3b) A user unauthorized to access a private dataset](#3b)
    - [(3c) A user that has an SSO account, but is not registered in our Admin](#3c)
    - [(3d) An anonymous user](#3d)

## NOTE
This notebook demonstrates how users will interact with the Auth feature in SPARCL. It is meant for internal-use only. Connection to sparc1 and sparclstage servers require VPN.

# Install the most recent version of the SPARCL client

In [2]:
#!pip install sparclclient==1.2.2b9

# Imports and setup

In [2]:
from sparcl.client import SparclClient
from getpass import getpass

In [3]:
client = SparclClient(url='https://sparc1.datalab.noirlab.edu')  ## PAT
#client = SparclClient(url='https://sparclstage.datalab.noirlab.edu')  ## STAGE
#client = SparclClient()  ## PROD
client

(sparclclient:1.2.2b9, api:11.0, https://sparc1.datalab.noirlab.edu/sparc, client_hash=, verbose=False, connect_timeout=1.1, read_timeout=5400.0)

In [4]:
auth_user = 'test_user_1@noirlab.edu'
unauth_user = 'test_user_2@noirlab.edu'
non_user = 'test_user_3@noirlab.edu'
usrpw = getpass()

 ········


#### Helper function

In [5]:
def count_records(res):
    
    count_sdss=0
    count_boss=0
    count_desi=0
    count_priv=0
    
    for r in res.records:
        if r.data_release == 'SDSS-DR16':
            count_sdss += 1
        elif r.data_release == 'BOSS-DR16':
            count_boss += 1
        elif r.data_release == 'DESI-EDR':
            count_desi += 1
        elif r.data_release == 'SDSS-DR17-test':
            count_priv += 1

    print(f"# of SDSS-DR16 records = {count_sdss}\n"
          f"# of BOSS-DR16 records = {count_boss}\n"
          f"# of DESI-EDR records = {count_desi}\n"
          f"# of SDSS-DR17-test records = {count_priv}")

<a class="anchor" id="loginout"></a>
# (1) Login, logout, and check authorized datasets

In [6]:
client.login?

Signature: client.login(email, password=None)
Docstring:
Login to the SPARCL service.

Args:
    email (:obj:`str`): User login email.

    password (:obj:`str`, optional): User SSO password.
        If not given, the output will prompt the user
        to enter in their SSO password.

Returns:
    None.

Example:
    >>>
    >> client = SparclClient()
    >> client.login('test_user@noirlab.edu', 'testpw')
    Logged in successfully with email='test_user@noirlab.edu'
File:      ~/anaconda3/lib/python3.8/site-packages/sparcl/client.py
Type:      method

In [7]:
client.logout?

Signature: client.logout()
Docstring:
Logout of the SPARCL service.

Args:
    None.

Returns:
    None.

Example:
    >>> client = SparclClient()
    >>> client.logout()
    Logged-out successfully.  Previously logged-in with email None.
File:      ~/anaconda3/lib/python3.8/site-packages/sparcl/client.py
Type:      method

<a class="anchor" id="1a"></a>
### (1a) A user authorized to access a private dataset

In [8]:
client.login(auth_user, usrpw)

Logged in successfully with email='test_user_1@noirlab.edu'


In [9]:
client.authorized

{'Loggedin_As': 'test_user_1@noirlab.edu',
 'Authorized_Datasets': {'BOSS-DR16',
  'DESI-EDR',
  'SDSS-DR16',
  'SDSS-DR17-test'}}

In [10]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_1@noirlab.edu.


<a class="anchor" id="1b"></a>
### (1b) A user unauthorized to access a private dataset

In [11]:
client.login(unauth_user, usrpw)

Logged in successfully with email='test_user_2@noirlab.edu'


In [12]:
client.authorized

{'Loggedin_As': 'test_user_2@noirlab.edu',
 'Authorized_Datasets': {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}}

In [13]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_2@noirlab.edu.


<a class="anchor" id="1c"></a>
### (1c) A user that has an SSO account, but is not registered in our Admin

In [14]:
client.login(non_user, usrpw)

Logged in successfully with email='test_user_3@noirlab.edu'


In [15]:
client.authorized

{'Loggedin_As': 'Anonymous',
 'Authorized_Datasets': {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}}

In [16]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_3@noirlab.edu.


<a class="anchor" id="1d"></a>
### (1d) An anonymous user

In [17]:
client.authorized

{'Loggedin_As': 'Anonymous',
 'Authorized_Datasets': {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}}

In [18]:
client.logout()

Logged-out successfully.  Previously logged-in with email None.


<a class="anchor" id="find"></a>
# (2) Auth with the `client.find()` method

In [19]:
# No dataset(s) specified in constraints
cons = {'spectype': ['GALAXY'],
        'redshift': [0.5, 0.9]}
# Private dataset specified in constraints
cons2 = {'spectype': ['STAR'],
        'data_release': ['SDSS-DR17-test', 'DESI-EDR']}
# Outfields
out = ['sparcl_id', 'data_release']

<a class="anchor" id="2a"></a>
### (2a) A user authorized to access a private dataset

In [20]:
client.login(auth_user, usrpw)

Logged in successfully with email='test_user_1@noirlab.edu'


In [21]:
# No dataset(s) specified in constraints.
auth_found = client.find(outfields=out,
                         constraints=cons,
                         limit=20000)
count_records(auth_found)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 19


In [22]:
# Private dataset specified in constraints.
auth_found2 = client.find(outfields=out,
                          constraints=cons2,
                          limit=20000)
count_records(auth_found2)

# of SDSS-DR16 records = 0
# of BOSS-DR16 records = 0
# of DESI-EDR records = 2096
# of SDSS-DR17-test records = 517


In [23]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_1@noirlab.edu.


<a class="anchor" id="2b"></a>
### (2b) A user unauthorized to access a private dataset

In [24]:
client.login(unauth_user, usrpw)

Logged in successfully with email='test_user_2@noirlab.edu'


In [25]:
# No dataset(s) specified in constraints.
unauth_found = client.find(outfields=out,
                           constraints=cons,
                           limit=20000)
count_records(unauth_found)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


In [26]:
# Private dataset specified in constraints.
# This cell should produce an "Access Not Allowed" error.
unauth_found2 = client.find(outfields=out,
                            constraints=cons2,
                            limit=20000)
count_records(unauth_found2)

AccessNotAllowed: [DSDENIED] test_user_2@noirlab.edu is declined access to datasets ['SDSS-DR17-test']

In [27]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_2@noirlab.edu.


<a class="anchor" id="2c"></a>
### (2c) A user that has an SSO account, but is not registered in our Admin

In [28]:
client.login(non_user, usrpw)

Logged in successfully with email='test_user_3@noirlab.edu'


In [29]:
# No dataset(s) specified in constraints.
non_found = client.find(outfields=out,
                        constraints=cons,
                        limit=20000)
count_records(non_found)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


In [30]:
# Private dataset specified in constraints.
# This cell should produce an "Access Not Allowed" error.
non_found2 = client.find(outfields=out,
                         constraints=cons2,
                         limit=20000)
count_records(non_found2)

AccessNotAllowed: [DSDENIED] ANONYMOUS is declined access to datasets ['SDSS-DR17-test']

In [31]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_3@noirlab.edu.


<a class="anchor" id="2d"></a>
### (2d) An anonymous user

In [32]:
# No dataset(s) specified in constraints.
anon_found = client.find(outfields=out,
                         constraints=cons,
                         limit=20000)
count_records(anon_found)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


In [33]:
# Private dataset specified in constraints.
# This cell should produce an "Access Not Allowed" error.
anon_found2 = client.find(outfields=out,
                          constraints=cons2,
                          limit=20000)
count_records(anon_found2)

AccessNotAllowed: [DSDENIED] ANONYMOUS is declined access to datasets ['SDSS-DR17-test']

<a class="anchor" id="retrieve"></a>
# (3) Auth with the `client.retrieve()` method

<a class="anchor" id="3a"></a>
### (3a) A user authorized to access a private dataset

In [34]:
client.login(auth_user, usrpw)

Logged in successfully with email='test_user_1@noirlab.edu'


In [35]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
auth_ret = client.retrieve(uuid_list=auth_found.ids,
                           include=out,
                           limit=20000)
count_records(auth_ret)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 19


In [36]:
# uuid_list includes IDs from private dataset, private dataset specified.
auth_ret2 = client.retrieve(uuid_list=auth_found2.ids,
                            include=out,
                            dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                            limit=20000)
count_records(auth_ret2)

# of SDSS-DR16 records = 0
# of BOSS-DR16 records = 0
# of DESI-EDR records = 2096
# of SDSS-DR17-test records = 517


In [37]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_1@noirlab.edu.


<a class="anchor" id="3b"></a>
### (3b) A user unauthorized to access a private dataset

In [38]:
client.login(unauth_user, usrpw)

Logged in successfully with email='test_user_2@noirlab.edu'


In [39]:
# uuid_list does not include IDs from private dataset, no dataset(s) specified.
unauth_ret = client.retrieve(uuid_list=unauth_found.ids,
                             include=out,
                             limit=20000)
count_records(unauth_ret)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


In [40]:
# uuid_list does not include IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
unauth_ret2 = client.retrieve(uuid_list=unauth_found.ids,
                              include=out,
                              dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                              limit=20000)
count_records(unauth_ret2)

AccessNotAllowed: [DSDENIED] test_user_2@noirlab.edu is declined access to datasets ['SDSS-DR17-test']

In [41]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
# This cell should only retrieve records from user-authorized datasets
# and output a UserWarning: Some UUIDs were not found.
unauth_ret3 = client.retrieve(uuid_list=auth_found.ids,
                              include=out,
                              limit=20000)
count_records(unauth_ret3)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


/var/folders/6j/v2lh3ssj13g6fs_lhk8nlps40000gp/T/ipykernel_52706/1687955117.py:4: UserWarning: Some UUIDs were not found. 4 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 4 unavailable uuids.; Some UUIDs were not found. 11 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 11 unavailable uuids.; Some UUIDs were not found. 3 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 3 unavailable uuids.; Some UUIDs were not found. 1 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.m

In [42]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
unauth_ret4 = client.retrieve(uuid_list=auth_found.ids,
                              include=out,
                              dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                              limit=20000)
count_records(unauth_ret4)

AccessNotAllowed: [DSDENIED] test_user_2@noirlab.edu is declined access to datasets ['SDSS-DR17-test']

In [43]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_2@noirlab.edu.


<a class="anchor" id="3c"></a>
### (3c) A user that has an SSO account, but is not registered in our Admin

In [44]:
client.login(non_user, usrpw)

Logged in successfully with email='test_user_3@noirlab.edu'


In [45]:
# uuid_list does not include IDs from private dataset, no dataset(s) specified.
non_ret = client.retrieve(uuid_list=non_found.ids,
                          include=out,
                          limit=20000)
count_records(non_ret)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


In [46]:
# uuid_list does not include IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
non_ret2 = client.retrieve(uuid_list=non_found.ids,
                           include=out,
                           dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                           limit=20000)
count_records(non_ret2)

AccessNotAllowed: [DSDENIED] ANONYMOUS is declined access to datasets ['SDSS-DR17-test']

In [47]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
# This cell should only retrieve records from user-authorized datasets
# and output a UserWarning: Some UUIDs were not found.
non_ret3 = client.retrieve(uuid_list=auth_found.ids,
                           include=out,
                           limit=20000)
count_records(non_ret3)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


/var/folders/6j/v2lh3ssj13g6fs_lhk8nlps40000gp/T/ipykernel_52706/2556312795.py:4: UserWarning: Some UUIDs were not found. 4 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 4 unavailable uuids.; Some UUIDs were not found. 11 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 11 unavailable uuids.; Some UUIDs were not found. 3 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 3 unavailable uuids.; Some UUIDs were not found. 1 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.m

In [48]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
non_ret4 = client.retrieve(uuid_list=auth_found.ids,
                           include=out,
                           dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                           limit=20000)
count_records(non_ret4)

AccessNotAllowed: [DSDENIED] ANONYMOUS is declined access to datasets ['SDSS-DR17-test']

In [49]:
client.logout()

Logged-out successfully.  Previously logged-in with email test_user_3@noirlab.edu.


<a class="anchor" id="3d"></a>
### (3d) An anonymous user

In [50]:
# uuid_list does not include IDs from private dataset, no dataset(s) specified.
anon_ret = client.retrieve(uuid_list=anon_found.ids,
                           include=out,
                           limit=20000)
count_records(anon_ret)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


In [51]:
# uuid_list does not include IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
anon_ret2 = client.retrieve(uuid_list=anon_found.ids,
                            include=out,
                            dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                            limit=20000)
count_records(anon_ret2)

AccessNotAllowed: [DSDENIED] ANONYMOUS is declined access to datasets ['SDSS-DR17-test']

In [52]:
# uuid_list includes IDs from private dataset, no dataset(s) specified.
# This cell should only retrieve records from user-authorized datasets
# and output a UserWarning: Some UUIDs were not found.
anon_ret3 = client.retrieve(uuid_list=auth_found.ids,
                            include=out,
                            limit=20000)
count_records(anon_ret3)

# of SDSS-DR16 records = 87
# of BOSS-DR16 records = 3071
# of DESI-EDR records = 1162
# of SDSS-DR17-test records = 0


/var/folders/6j/v2lh3ssj13g6fs_lhk8nlps40000gp/T/ipykernel_52706/134315787.py:4: UserWarning: Some UUIDs were not found. 4 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 4 unavailable uuids.; Some UUIDs were not found. 11 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 11 unavailable uuids.; Some UUIDs were not found. 3 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.missing()" to get a list of the 3 unavailable uuids.; Some UUIDs were not found. 1 out of the 500 requested uuids have no records available in the SPARCL database associated with DataSets {'BOSS-DR16', 'DESI-EDR', 'SDSS-DR16'}.Use "client.mi

In [53]:
# uuid_list includes IDs from private dataset, private dataset specified.
# This cell should produce an "Access Not Allowed" error.
anon_ret4 = client.retrieve(uuid_list=auth_found.ids,
                            include=out,
                            dataset_list=['SDSS-DR17-test', 'DESI-EDR'],
                            limit=20000)
count_records(anon_ret4)

AccessNotAllowed: [DSDENIED] ANONYMOUS is declined access to datasets ['SDSS-DR17-test']